# PREPARACIÓN DE LOS DATOS

En este apartado se tratarán todas las URLs realizando las siguientes tres operaciones:

1. __Limpieza__ de los __datos__<br>
2. __Equilibrado__ de las __clases__<br>
3. __Separación__ en conjuntos de __entrenamiento__ y __prueba__<br>

Posteriormente se almacenará cada una de los subconjuntos en dos dataframes diferentes.

- Importamos las _librerías_ necesarias:

In [1]:
import re
import time
import numpy as np

import random
import tldextract
import ipaddress
from urllib.parse import urlparse
import pandas as pd

from urllib.request import urlopen
import requests 
from bs4 import BeautifulSoup

- __Directorios__ utilizados:

In [2]:
import os
PROJECT_ROOT_PATH = "."
DATASETS_PATH = PROJECT_ROOT_PATH + os.sep + "datasets"
FINAL_DATASETS_PATH = PROJECT_ROOT_PATH + os.sep + "final_datasets"

- Con la siguiente función leemos el fichero CSV generado en el paso 1 y lo almacenamos en un _DataFrame_:<br>

In [3]:
def load_data(filename, separator, folder, path=FINAL_DATASETS_PATH):
    file_path = os.path.join(path, folder + os.sep + filename)
    return pd.read_csv(file_path, sep=separator)

In [4]:
df = load_data("1_urls_dataset.csv",',', "1_recogida_datos")

***
## 1. Limpieza de los datos<br>
Se tratarán aquellos valores que sean erróneos. Para ello se realizarán los siguientes tratamientos:
1. Agregación de protocolo http en caso de no tener protocolo
2. Eliminación de URLs duplicadas
3. Eliminación de URLs malformadas
4. Conversión a minúsculas

***
### Agregación protocolo http<br>

- __Comprobamos__ si todas las URLs contienen protocolo. En caso contrario añadimos el protocolo __http__:<br>

In [5]:
def contains_scheme(url):
    if not url.startswith('http://') and not url.startswith('https://'):
        return False
    return True

- Comprobamos que existen __283 URLs__ que __no contienen protocolo http o https__:<br>

In [6]:
df['scheme'] = df['url'].apply(contains_scheme)
df.groupby(['scheme']).count()

,url,label
scheme,,
False,283,283
True,716958,716958


- Eliminamos la columna _scheme_:

In [7]:
df = df.drop(['scheme'], axis=1)

### Eliminación de URLs duplicadas<br>

__Eliminamos__ los posibles datos __duplicados__ existentes en el Dataframe "df":<br>

In [8]:
df.drop_duplicates(keep='first', inplace=True)
df = df.sample(frac=1).reset_index(drop=True)
df.groupby(['label']).count()

,url
label,
0,345734
1,362608


- Podemos comprobar que existían __8899 URLs duplicadas__.

In [9]:
print((371503 + 345738) - len(df))

8899


### Eliminación de URLs malformadas <br>

- __Eliminación__ de todas aquellas URLs que no sigan el formato de formación correcto de las URLs:<br>

In [10]:
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError

def url_is_valid(url):
    is_valid = URLValidator()
    try:
        is_valid(url)
        return True
    except ValidationError:
        return False

- Comprobamos que existen __596 URLs malformadas__:<br>

In [11]:
df['valid'] = df['url'].apply(url_is_valid)
df.groupby(['valid']).count()

,url,label
valid,,
False,596,596
True,707746,707746


In [12]:
df = df.drop(df[df.valid == False].index)
df.groupby(['valid']).count()

,url,label
valid,,
True,707746,707746


In [13]:
#Eliminamos la columna 'valid'
df = df.drop(['valid'], axis=1)

### Conversión a minúsculas (lowercase) <br>

- __Convertimos__ todos los carácteres letra (A-Za-z) a minúscula que componen cada una de las diferentes URLs:<br>

In [14]:
#Función para comprobar que una URL contiene carácteres en mayúscula (uppercase)
def contains_uppercase(url):
    if (any(c.isupper() for c in url)):
        return True
    return False

- Comprobamos que existen __304902 URLs__ que __contienen carácteres__ en __mayúscula (uppercase)__:<br>

In [15]:
df['uppercase'] = df['url'].apply(contains_uppercase)
df.groupby(['uppercase']).count()

,url,label
uppercase,,
False,402844,402844
True,304902,304902


In [16]:
#Función para convertir a minúsculas todas las URLs
def to_lowercase(url):
    return url.lower()

In [17]:
df['url'] = df['url'].apply(to_lowercase)
df['uppercase'] = df['url'].apply(contains_scheme)
df.groupby(['uppercase']).count()

,url,label
uppercase,,
False,21,21
True,707725,707725


- Eliminamos la columna _uppercase_:

In [18]:
df = df.drop(['uppercase'], axis=1)

***
## 2. Separación de los datos<br>

- Importamos la librería train_test_split para dividir los datos:<br>

In [19]:
from sklearn.model_selection import train_test_split

- Reservamos el 30% de los datos del conjunto inicial para el conjunto de prueba. A continuación calculamos el tamaño de el subconjunto de prueba:

In [20]:
print("Tamaño dataset inicial: " + str(len(df)))
test_size = round(len(df) * 0.3)
print("- Tamaño conjunto de entrenamiento: " + str(len(df)-test_size))
print("- Tamaño conjunto de prueba: " + str(test_size))

Tamaño dataset inicial: 707746
- Tamaño conjunto de entrenamiento: 495422
- Tamaño conjunto de prueba: 212324


- Dividimos los datos. Mediante el parámetro _stratify_ especificamos que cada subconjunto contenga la misma cantidad de datos de ambas clases especificándole que tome como referencia la columna _label_:<br>

In [21]:
train_df, test_df = train_test_split(df, test_size=test_size, random_state=42, stratify=df[['label']])
print("- Tamaño conjunto entrenamiento: " + str(len(train_df)))
print("- Tamaño conjunto prueba: " + str(len(test_df)))

- Tamaño conjunto entrenamiento: 495422
- Tamaño conjunto prueba: 212324


- El conjunto de entrenamiento posee prácticamente el mismo número de URLs maliciosas como no maliciosas:<br>

In [22]:
train_df.groupby(['label']).count()

,url
label,
0,241989
1,253433


- Ocurre lo mismo con el subconjunto de prueba:<br>

In [23]:
test_df.groupby(['label']).count()

,url
label,
0,103709
1,108615


***
## Guardamos los datos<br>
- __Guardamos__ el contenido del _DataFrame_ final para realizar la extracción de características como siguiente paso:<br>

In [24]:
def save_data(dataframe, filename, separator, folder, path=FINAL_DATASETS_PATH):
    file_path = os.path.join(path, folder + os.sep + filename)
    dataframe.to_csv(file_path, sep=separator, index=False)

In [25]:
#Train DataFrame
save_data(train_df, "3_train_dataset.csv", ',', "3_preparacion_datos")

#Test DataFrame
save_data(test_df, "3_test_dataset.csv", ',', "3_preparacion_datos")